In [1]:
import numpy as np
import pandas as pd

In [2]:
x = np.random.randint(5, size=[10,4]) +10
y = np.random.randint(5, size=[10,4])

In [3]:
def pairwiseDist(x, y=None):
    if y is None:
        y = x
    return np.sum((x[:,None]-y)**2,axis=2)**0.5

def arrayDist(x,y):
    return np.sum((x-y)**2,axis=1)**0.5

def errRates(pred, actual):
    return (actual!=pred).sum() / pred.size # return error rate

In [49]:
pairwiseDist(x[:3], x)

array([[ 0.        ,  4.24264069,  4.12310563,  3.31662479,  5.09901951,
         4.24264069,  4.        ,  3.74165739,  3.16227766,  4.24264069],
       [ 4.24264069,  0.        ,  1.        ,  1.73205081,  4.47213595,
         3.74165739,  4.24264069,  2.        ,  2.44948974,  0.        ],
       [ 4.12310563,  1.        ,  0.        ,  1.41421356,  3.87298335,
         3.        ,  4.58257569,  2.64575131,  3.        ,  1.        ]])

In [43]:
def initMeans(data, n, algo=1):
    if algo==1: # choose n random points
        idx = np.random.choice(range(data.shape[0]), n, False) # no replace
        out = data[idx,]
    if algo==2: # random means points (0,1)
        out = np.random.random([n,data.shape[1]])
        
    return out

def updateMeans(data, means, k):
    ## Assign each pt to the mean for which it has the shortest distance
    tmpDist = pairwiseDist(means,data) # dist between means and all data pts
    #print(tmpDist)
    minDist = tmpDist.argmin(axis=0) # group where data pt has min distance

    ## Calculate new means to be centroid of all the points in the group
    newMeans = np.zeros([k,data.shape[1]]) # new mean points
    for n,x in enumerate(means): # loop over all clusters
        tmp = np.vstack( (data[minDist==n,],x) ) # concat data pt and centroid
        newMeans[n] = tmp.mean(axis=0) # new mean = centroid of all pts 
    
    return newMeans,minDist

################################################################################

def kMeans(data, k, trace=False, initAlgo=1):
    means = initMeans(data, k, initAlgo) # initialize mean points
    for n in range(5):
        if trace:
            print(means)
        means,grpIdx = updateMeans(data, means, k)
    return means,grpIdx # return final centroids and labels

In [40]:
updateMeans(x, x[:3], 3)

[[ 0.          4.24264069  4.12310563  3.31662479  5.09901951  4.24264069
   4.          3.74165739  3.16227766  4.24264069]
 [ 4.24264069  0.          1.          1.73205081  4.47213595  3.74165739
   4.24264069  2.          2.44948974  0.        ]
 [ 4.12310563  1.          0.          1.41421356  3.87298335  3.
   4.58257569  2.64575131  3.          1.        ]]
[[ 0.          4.24264069  4.12310563  3.31662479  5.09901951  4.24264069
   4.          3.74165739  3.16227766  4.24264069]
 [ 4.24264069  0.          1.          1.73205081  4.47213595  3.74165739
   4.24264069  2.          2.44948974  0.        ]
 [ 4.12310563  1.          0.          1.41421356  3.87298335  3.
   4.58257569  2.64575131  3.          1.        ]]


(array([[ 11.66666667,  12.66666667,  13.33333333,  12.66666667],
        [ 11.4       ,  11.2       ,  10.6       ,  13.4       ],
        [ 11.8       ,  10.8       ,  10.6       ,  11.2       ]]),
 array([0, 1, 2, 2, 2, 2, 0, 1, 1, 1]))

In [47]:
z = np.vstack([x,y])

kMeans(z, 2, True, 2)

[[ 0.46323279  0.51412241  0.93678707  0.29727731]
 [ 0.12199274  0.58031046  0.15226517  0.46345296]]
[[ 7.35906849  7.13967347  7.38537706  7.51651541]
 [ 1.28049819  0.89507761  0.03806629  2.36586324]]
[[ 11.39627895  11.01269759  10.8532161   11.95604686]
 [  1.84368165   1.62682524   1.82164239   2.03326029]]
[[ 11.76329809  11.36479069  11.16847419  12.35964062]
 [  1.89488015   1.69334775   1.98378567   2.00302366]]
[[ 11.79666346  11.39679915  11.19713402  12.39633097]
 [  1.89953456   1.69939525   1.99852597   2.00027488]]


array([[ 11.79969668,  11.39970901,  11.19973946,  12.39966645],
       [  1.89995769,   1.69994502,   1.999866  ,   2.00002499]])

In [ ]:
def ForSelect(data, classVec):
    k = len(np.unique(classVec))
    err = np.Infinity # largest error possible
    
    while 

In [50]:
def Silhouette(data, labels):
    nData = data.shape[0]
    
    ################################################################################
    uniqKs,invIdx = np.unique(labels, return_inverse=True) # unique K and inv indx
    allDists = pairwiseDist(data) # pre-calc all pairwise dist for memoization
    grpIdx = pd.Series(labels).groupby(labels).groups.items() # idx for each group
    
    aVals,bVals = np.zeros(nData),np.zeros(nData) # pre-allocate a's and b's
    for grp,idx in grpIdx: # loop over all groups
        aVals[idx] = allDists[idx,idx].mean(axis=0) # a's for all data pts in class
        
        # loop over all groups that's not the current gruop
        tmp = np.zeros([len(idx),len(grpIdx)-1]) # tmp for all b's for curr grp
        for n,(_,outIdx) in enumerate([x for x in grpIdx if x[0]!=grp]):
            tmp[n,] = allDists[idx,outIdx].mean(axis=0) # mean dist of out-group
        print(tmp)
        print(idx)
        bVals[idx] = np.min(tmp, axis=1)
        
    return (bVals-aVals)/np.maximum(aVals,bVals) # calc and return silhouette coeff


In [57]:
testX = np.array([[1,1,1],[10,10,10],[11,11,11],
                  [2,2,1],[1.5,1.5,1.5],[12,12,12]]) + np.random.random([6,3])/10
testY = np.array([1,2,2,1,1,2])
print(testX)

[[ 1.08099425  1.022068    1.0644936 ]
 [10.08191943 10.04202835 10.05573231]
 [11.02779647 11.08240859 11.04466729]
 [ 2.0020292   2.09433134  1.0456484 ]
 [ 1.5609202   1.54609669  1.5181049 ]
 [12.06231991 12.07028084 12.05602924]]


In [58]:
Silhouette(testX,testY)

[[16.67073956]
 [ 0.        ]
 [ 0.        ]]
Int64Index([0, 3, 4], dtype='int64')
[[16.67073956]
 [ 0.        ]
 [ 0.        ]]
Int64Index([1, 2, 5], dtype='int64')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


array([ 1.,  1., nan, nan, nan, nan])

In [55]:
np.random.random([5,4])/10

array([[0.04017776, 0.06630331, 0.09935284, 0.09698802],
       [0.06445735, 0.05550504, 0.0328987 , 0.08139582],
       [0.01785217, 0.02968005, 0.02322455, 0.00259945],
       [0.04867259, 0.07075442, 0.05013823, 0.02000517],
       [0.09423152, 0.03031394, 0.09158001, 0.01274519]])

## Sources

- [A Modified k-means Algorithm to Avoid Empty Clusters](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.3148&rep=rep1&type=pdf)